# Sentiment Analysis - Twitter, IMDB, YELP

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import copy

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/BSAN Projects/NLP_735_Team1/im_Train.csv', encoding='utf-8')
test_df = pd.read_csv('/content/drive/MyDrive/BSAN Projects/NLP_735_Team1/im_Test.csv', encoding='utf-8')
val_df = pd.read_csv('/content/drive/MyDrive/BSAN Projects/NLP_735_Team1/im_valid.csv', encoding='utf-8')

In [5]:
train_df.columns

Index(['text', 'label'], dtype='object')

In [6]:
test = copy.deepcopy(test_df)
train = copy.deepcopy(train_df)
valid = copy.deepcopy(val_df)

# text pre process

In [7]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 38.1 MB/s eta 0:00:00


In [9]:
import contractions
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def text_preprocess(text):
    # Convert input to string
    text = str(text)
    
    # Expand contractions
    text = contractions.fix(text)

    # Remove punctuation and stress marks using Unicode
    text = ''.join(c for c in text if not unicodedata.category(c).startswith('P'))
    text = ''.join(c for c in text if not unicodedata.category(c).startswith('Sk'))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Lemmatize words using NLTK
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove stopwords, excluding negation words
    negation_words = {'not', 'no', 'n\'t', 'never', 'none', 'nobody', 'nothing', 'neither', 'nowhere', 'cant', 'can\'t', 'dont', 'don\'t'}
    stop_words = set(stopwords.words('english')) - negation_words
    filtered_tokens = [token for token in lemmatized_tokens if isinstance(token, str) and token.lower() not in stop_words]

    # Reconstruct the processed text
    processed_text = ' '.join(filtered_tokens)

    return processed_text

# Example usage
input_text = "This is a sample sentence, which won't be ignored."
output_text = text_preprocess(input_text)
print(output_text)

sample sentence not ignored


In [10]:
train['preprocessed_text'] = train['text'].apply(text_preprocess) 
test['preprocessed_text'] = test['text'].apply(text_preprocess) 
valid['preprocessed_text'] = valid['text'].apply(text_preprocess) 

In [11]:
imdb = pd.concat([train,test,valid])

In [12]:
imdb.head()

,text,label,preprocessed_text
0,I grew up (b. 1965) watching and loving the Th...,0,grew b 1965 watching loving Thunderbirds mate ...
1,"When I put this movie in my DVD player, and sa...",0,put movie DVD player sat coke chip expectation...
2,Why do people who do not know what a particula...,0,people not know particular time past wa like f...
3,Even though I have great interest in Biblical ...,0,Even though great interest Biblical movie wa b...
4,Im a die hard Dads Army fan and nothing will e...,1,die hard Dads Army fan nothing ever change got...


# Nested CV

In [13]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score

def train_and_evaluate(X_train_all, y_train_all, X_test_all, y_test_all):
    skf_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    skf_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    outer_fold = 5

    for train_index, test_index in skf_outer.split(X_test_all, y_test_all):
        print('Outer fold', outer_fold)

        X_test, y_test = np.array(X_test_all)[test_index], np.array(y_test_all)[test_index]

        inner_fold = 10
        best_inner_score = 0
        best_inner_params = None

        for sub_train_index, dev_index in skf_inner.split(X_train_all, y_train_all):
            print('Inner fold', inner_fold)

            X_train, X_dev = np.array(X_train_all)[sub_train_index], np.array(X_train_all)[dev_index]
            y_train, y_dev = np.array(y_train_all)[sub_train_index], np.array(y_train_all)[dev_index]

            clf = Pipeline([('vect', TfidfVectorizer(smooth_idf=True)),
                            ('clf', LogisticRegression(penalty='l2', solver='liblinear', max_iter=1500, random_state=0))])

            for C in [10]:  # Tune the 'C' hyperparameter
                clf.set_params(clf__C=C)
                clf.fit(X_train, y_train)
                y_dev_pred = clf.predict(X_dev)

                dev_f1_score = f1_score(y_dev, y_dev_pred, average='macro')
                print(f"Inner fold {inner_fold}, C={C}, F1 Score: {dev_f1_score}")

                if dev_f1_score > best_inner_score:
                    best_inner_score = dev_f1_score
                    best_inner_params = {'clf__C': C}

            inner_fold += 1

        print(f"Best inner params for outer fold {outer_fold}: {best_inner_params}")

        # Train the model using the best inner params on the entire train set of the current outer fold
        clf.set_params(**best_inner_params)
        clf.fit(X_train_all, y_train_all)

        y_test_pred = clf.predict(X_test)
        print(classification_report(y_test, y_test_pred))

        outer_fold += 1

# Load your dataset here
# For example, let's assume you have a dataset called `tweets` with preprocessed_text and sentiment columns
X = imdb['preprocessed_text']
y = imdb['label']

# Split the dataset into train and test sets using train_test_split
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Call the train_and_evaluate function with the input data
train_and_evaluate(X_train_all, y_train_all, X_test_all, y_test_all)


Outer fold 5
Inner fold 10
Inner fold 10, C=10, F1 Score: 0.9027058965798582
Inner fold 11
Inner fold 11, C=10, F1 Score: 0.8904213056902488
Inner fold 12
Inner fold 12, C=10, F1 Score: 0.8935690631981597
Inner fold 13
Inner fold 13, C=10, F1 Score: 0.8934257526484781
Inner fold 14
Inner fold 14, C=10, F1 Score: 0.8979919975762998
Best inner params for outer fold 5: {'clf__C': 10}
              precision    recall  f1-score   support

           0       0.91      0.88      0.90      1500
           1       0.88      0.92      0.90      1500

    accuracy                           0.90      3000
   macro avg       0.90      0.90      0.90      3000
weighted avg       0.90      0.90      0.90      3000

Outer fold 6
Inner fold 10
Inner fold 10, C=10, F1 Score: 0.9027058965798582
Inner fold 11
Inner fold 11, C=10, F1 Score: 0.8904213056902488
Inner fold 12
Inner fold 12, C=10, F1 Score: 0.8935690631981597
Inner fold 13
Inner fold 13, C=10, F1 Score: 0.8934257526484781
Inner fold 14
Inner 

# Trainable embed and Pre-trained GloVe

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# Assuming you have a dataset called imdb with 'preprocessed_text' and 'label' columns
X = imdb['preprocessed_text']
y = imdb['label']

# Initialize the LabelBinarizer and fit it on the labels
lb = LabelBinarizer()
lb.fit(y)

# Transform the labels into one-hot encoded format
y_one_hot = lb.transform(y)

# Tokenize the text data
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

# Convert y_train and y_test to one-hot encoding
num_classes = 2  # Replace this with the number of unique labels in your dataset
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Update vocab_size for the model
vocab_size = len(tokenizer.word_index) + 1

# Create the Keras model with the same architecture you provided earlier
embedding_dim = 100
lstm_units = 128

model_trainable = Sequential()
model_trainable.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model_trainable.add(LSTM(lstm_units))
model_trainable.add(Dense(2))
model_trainable.add(BatchNormalization())
model_trainable.add(Activation('sigmoid'))

# Compile the model with the specified optimizer settings
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)

model_trainable.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model_trainable.summary()

# Early stopping based on validation accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

# Train the model
history_trainable = model_trainable.fit(X_train, y_train_one_hot, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

test_loss, test_accuracy = model_trainable.evaluate(X_test, y_test_one_hot, batch_size=64)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

# Get predicted probabilities for the test set
y_pred_probs = model_trainable.predict(X_test)

# Convert probabilities to predicted classes
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Convert one-hot encoded y_test back to class labels
y_test_classes = np.argmax(y_test_one_hot, axis=1)

# Print the classification report
print(classification_report(y_test_classes, y_pred_classes))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          16119000  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
 batch_normalization (BatchN  (None, 2)                8         
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                                 
Total params: 16,236,514
Trainable params: 16,236,510
Non-trainable params: 4
____________________________________________

# GloVe

# Load GloVe embeddings and create embedding matrix

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have a dataset called imdb with 'preprocessed_text' and 'label' columns
X = imdb['preprocessed_text']
y = imdb['label']

# Initialize the LabelBinarizer and fit it on the labels
lb = LabelBinarizer()
lb.fit(y)

# Transform the labels into one-hot encoded format
y_one_hot = lb.transform(y)

# Tokenize the text data
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

# Convert y_train and y_test to one-hot encoding
num_classes = 2  # Replace this with the number of unique labels in your dataset
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Update vocab_size for the model
vocab_size = len(tokenizer.word_index) + 1

# Load GloVe embeddings
glove_embeddings = {}
with open('/content/drive/MyDrive/BSAN Projects/glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        glove_embeddings[word] = vector

# Create GloVe embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, index in tokenizer.word_index.items():
    embedding_vector = glove_embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

# Create the Keras model with the same architecture you provided earlier
embedding_dim = 100
lstm_units = 128

model_trainable = Sequential()
model_trainable.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length, weights=[embedding_matrix], trainable=False))
model_trainable.add(LSTM(lstm_units))
model_trainable.add(Dense(2))
model_trainable.add(BatchNormalization())
model_trainable.add(Activation('sigmoid'))

# Compile the model with the specified optimizer settings
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)

model_trainable.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model_trainable.summary()

# Early stopping based on validation accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

# Train the model
history_trainable = model_trainable.fit(X_train, y_train_one_hot, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model_trainable.evaluate(X_test, y_test_one_hot, batch_size=64)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

from sklearn.metrics import classification_report

# Get predicted probabilities for the test set
y_pred_probs = model_trainable.predict(X_test)

# Convert probabilities to predicted classes
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Convert one-hot encoded y_test back to class labels
y_test_classes = np.argmax(y_test_one_hot, axis=1)

# Print the classification report
print(classification_report(y_test_classes, y_pred_classes))


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          16119000  
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
 batch_normalization_1 (Batc  (None, 2)                8         
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 2)                 0         
                                                                 
Total params: 16,236,514
Trainable params: 117,510
Non-trainable params: 16,119,004
____________________________________

# Trainable embed and Avg. Pooling

In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Activation, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# Assuming you have a dataset called imdb with 'preprocessed_text' and 'label' columns
X = imdb['preprocessed_text']
y = imdb['label']

# Initialize the LabelBinarizer and fit it on the labels
lb = LabelBinarizer()
lb.fit(y)

# Transform the labels into one-hot encoded format
y_one_hot = lb.transform(y)

# Tokenize the text data
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

# Convert y_train and y_test to one-hot encoding
num_classes = 2  # Replace this with the number of unique labels in your dataset
y_train_one_hot = to_categorical(y_train, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test, num_classes=num_classes)

# Update vocab_size for the model
vocab_size = len(tokenizer.word_index) + 1

# Create the Keras model with the same architecture you provided earlier
embedding_dim = 100
lstm_units = 128

model_trainable = Sequential()
model_trainable.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model_trainable.add(LSTM(lstm_units, return_sequences=True))
model_trainable.add(GlobalAveragePooling1D())
model_trainable.add(Dense(2))
model_trainable.add(BatchNormalization())
model_trainable.add(Activation('sigmoid'))

# Compile the model with the specified optimizer settings
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)

model_trainable.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model_trainable.summary()

# Early stopping based on validation accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

# Train the model
history_trainable = model_trainable.fit(X_train, y_train_one_hot, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

test_loss, test_accuracy = model_trainable.evaluate(X_test, y_test_one_hot, batch_size=64)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

# Get predicted probabilities for the test set
y_pred_probs = model_trainable.predict(X_test)

# Convert probabilities to predicted classes
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Convert one-hot encoded y_test back to class labels
y_test_classes = np.argmax(y_test_one_hot, axis=1)

# Print the classification report
print(classification_report(y_test_classes, y_pred_classes))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          16119000  
                                                                 
 lstm_2 (LSTM)               (None, 100, 128)          117248    
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 2)                 258       
                                                                 
 batch_normalization_2 (Batc  (None, 2)                8         
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 2)                